In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import tensorflow as tf


import numpy as np
import random
random.seed(72)

In [0]:
df=pd.read_csv("/content/gdrive/My Drive/data/emotion1channel.csv")
df

,attention,meditation,delta,theta,lowAplha,highAlpha,lowBeta,highBeta,lowGamma,highGamma,class
0,88,17,1290697,18187,6345,7462,7266,4278,2589,1228,7
1,88,17,105432,21344,8323,4496,4784,12071,2024,1123,7
2,83,29,732143,37527,48422,10286,9499,6050,2362,4157,7
3,80,26,21265,24517,7051,1790,9106,5771,1977,3265,7
4,69,20,349390,145647,10068,21707,11878,19883,9971,6592,7
...,...,...,...,...,...,...,...,...,...,...,...
13364,66,61,36288,10942,15819,4781,10488,11690,7245,2287,1
13365,63,81,434483,16548,25760,10239,7332,3822,9214,5527,1
13366,61,91,11198,21200,18905,20228,4850,12856,5412,7044,1
13367,56,88,537338,31723,1915,13033,10876,8500,7453,3461,1


In [0]:
df.isnull().sum()

attention     0
meditation    0
delta         0
theta         0
lowAplha      0
highAlpha     0
lowBeta       0
highBeta      0
lowGamma      0
highGamma     0
class         0
dtype: int64

In [0]:
#df=x=df[df['class']<5 ]
df["class"].unique()



array([7, 5, 3, 4, 0, 2, 1, 6])

In [0]:
df.loc[df["class"] == 1, "class"] = 0
df.loc[df["class"] == 2, "class"] = 1
df.loc[df["class"] == 3, "class"] = 1
df.loc[df["class"] == 4, "class"] = 2
df.loc[df["class"] == 5, "class"] = 2
df.loc[df["class"] == 6, "class"] = 3
df.loc[df["class"] == 7, "class"] = 3
df.head()

,attention,meditation,delta,theta,lowAplha,highAlpha,lowBeta,highBeta,lowGamma,highGamma,class
0,88,17,1290697,18187,6345,7462,7266,4278,2589,1228,3
1,88,17,105432,21344,8323,4496,4784,12071,2024,1123,3
2,83,29,732143,37527,48422,10286,9499,6050,2362,4157,3
3,80,26,21265,24517,7051,1790,9106,5771,1977,3265,3
4,69,20,349390,145647,10068,21707,11878,19883,9971,6592,3


In [0]:
#x=df.drop(["class"]  ,axis=1)
x=df.drop(["class","attention","meditation"]  ,axis=1)
#x=df
x

,delta,theta,lowAplha,highAlpha,lowBeta,highBeta,lowGamma,highGamma
0,1290697,18187,6345,7462,7266,4278,2589,1228
1,105432,21344,8323,4496,4784,12071,2024,1123
2,732143,37527,48422,10286,9499,6050,2362,4157
3,21265,24517,7051,1790,9106,5771,1977,3265
4,349390,145647,10068,21707,11878,19883,9971,6592
...,...,...,...,...,...,...,...,...
13364,36288,10942,15819,4781,10488,11690,7245,2287
13365,434483,16548,25760,10239,7332,3822,9214,5527
13366,11198,21200,18905,20228,4850,12856,5412,7044
13367,537338,31723,1915,13033,10876,8500,7453,3461


In [0]:
y = df.loc[:,'class'].values
print(y)
print(x.values)
x=x.values


[3 3 3 ... 0 0 0]
[[1290697   18187    6345 ...    4278    2589    1228]
 [ 105432   21344    8323 ...   12071    2024    1123]
 [ 732143   37527   48422 ...    6050    2362    4157]
 ...
 [  11198   21200   18905 ...   12856    5412    7044]
 [ 537338   31723    1915 ...    8500    7453    3461]
 [ 534966   54906   30588 ...   17227   14293    3204]]


In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
from keras.utils import to_categorical
y = to_categorical(y)
y

Using TensorFlow backend.


array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)

In [0]:
x_train = np.reshape(x_train, (x_train.shape[0],1,x.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,x.shape[1]))


In [0]:
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(1,8),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="sigmoid"))
model.add(Dropout(0.5))
#model.add(LSTM(100,return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(50))
#model.add(Dropout(0.2))
model.add(Dense(4, activation='sigmoid'))
from keras.optimizers import SGD
model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 64)             18688     
_________________________________________________________________
dropout (Dropout)            (None, 1, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 132       
Total params: 31,236
Trainable params: 31,236
Non-trainable params: 0
_________________________________________________________________


In [0]:
#history = model.fit(x_train, y_train, validation_split=0.33, epochs=50, batch_size=10, verbose=0)

history = model.fit(x_train, y_train, epochs = 10, validation_data= (x_test, y_test))
score, acc = model.evaluate(x_test, y_test)


Epoch 1/10
316/316 [==============================] - 1s 4ms/step - loss: 1.3410 - accuracy: 0.3936 - val_loss: 1.2226 - val_accuracy: 0.5063
Epoch 2/10
316/316 [==============================] - 1s 3ms/step - loss: 1.2449 - accuracy: 0.5006 - val_loss: 1.2185 - val_accuracy: 0.5063
Epoch 3/10
316/316 [==============================] - 1s 3ms/step - loss: 1.2356 - accuracy: 0.5020 - val_loss: 1.2171 - val_accuracy: 0.5063
Epoch 4/10
316/316 [==============================] - 1s 3ms/step - loss: 1.2314 - accuracy: 0.5018 - val_loss: 1.2158 - val_accuracy: 0.5063
Epoch 5/10
316/316 [==============================] - 1s 3ms/step - loss: 1.2304 - accuracy: 0.5018 - val_loss: 1.2157 - val_accuracy: 0.5063
Epoch 6/10
316/316 [==============================] - 1s 3ms/step - loss: 1.2292 - accuracy: 0.5020 - val_loss: 1.2150 - val_accuracy: 0.5063
Epoch 7/10
316/316 [==============================] - 1s 3ms/step - loss: 1.2269 - accuracy: 0.5017 - val_loss: 1.2148 - val_accuracy: 0.5063
Epoch 

In [0]:
print('Test score:', score)
print('Test accuracy:', acc)


Test score: 1.2140363454818726
Test accuracy: 0.506344199180603


In [0]:

from sklearn.metrics import accuracy_score
pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
print(expected_classes.shape)
print(predict_classes.shape)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Training Accuracy: {correct}")

(2522,)
(2522,)
Training Accuracy: 0.5063441712926249


In [0]:
print(predict_classes.shape)
print(expected_classes.shape)


(2522,)
(2522,)
